In [1]:
from torch.autograd import Variable
import torch.nn.functional as F
import torch.nn as nn
import torch
import torchvision
from torchvision import datasets, transforms
import torch.utils.data as data
import torchvision.models as models
import matplotlib.image as pli
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
from PIL import Image

In [2]:
my_transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

file_path = './TinyImageNet'

In [3]:
label_num = 100

class ImageSet(data.Dataset):
    def __init__(self):
        self.length = 10000

    def __getitem__(self, index):
        img = Image.open(f'{file_path}/test/{index}.jpg')
        # img.show()
        img = my_transform(img)
        # print(img.size())
        # print(aaa)
        return index, img

    def __len__(self):
        return self.length

In [4]:
test_loader = data.DataLoader(ImageSet(), batch_size=100, shuffle=False)

In [5]:
# 加载模型， 请谨慎操作， 会覆盖在内存中的模型
convNet = models.resnet18()
convNet.conv1 = nn.Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
convNet.maxpool = nn.Sequential()
convNet.layer4 = nn.Sequential()
convNet.fc = nn.Linear(256,label_num)
convNet.load_state_dict(torch.load('./ConvNet.model'))
convNet.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): Sequential()
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1),

In [14]:
convNet.eval()
device = torch.device("cuda")
convNet = convNet.to(device)
index_list = torch.zeros((0)).int()
index_list = index_list.to(device)
predict_list = torch.zeros((0)).int()
predict_list = predict_list.to(device)
for i, (index, images) in enumerate(test_loader):
    index = index.to(device)
    images = images.to(device)
    outputs = convNet(images)
    predict = torch.argmax(F.softmax(outputs, dim=1), dim=1)
    print(f"i = {i}, index = {index[0]}:{index[-1]} predict = {predict[0]}:{predict[-1]}")
    index_list = torch.cat((index_list,index.int()))
    predict_list = torch.cat((predict_list,predict.int()))
    # print(index_list)
    # print(predict_list)

i = 0, index = 0:99 predict = 84:22
i = 1, index = 100:199 predict = 95:14
i = 2, index = 200:299 predict = 68:75
i = 3, index = 300:399 predict = 78:61
i = 4, index = 400:499 predict = 36:28
i = 5, index = 500:599 predict = 40:96
i = 6, index = 600:699 predict = 35:80
i = 7, index = 700:799 predict = 62:6
i = 8, index = 800:899 predict = 3:99
i = 9, index = 900:999 predict = 73:44
i = 10, index = 1000:1099 predict = 92:38
i = 11, index = 1100:1199 predict = 14:57
i = 12, index = 1200:1299 predict = 81:95
i = 13, index = 1300:1399 predict = 78:54
i = 14, index = 1400:1499 predict = 25:7
i = 15, index = 1500:1599 predict = 79:57
i = 16, index = 1600:1699 predict = 14:73
i = 17, index = 1700:1799 predict = 32:79
i = 18, index = 1800:1899 predict = 21:38
i = 19, index = 1900:1999 predict = 80:40
i = 20, index = 2000:2099 predict = 53:76
i = 21, index = 2100:2199 predict = 93:89
i = 22, index = 2200:2299 predict = 60:52
i = 23, index = 2300:2399 predict = 28:45
i = 24, index = 2400:2499 pr

In [15]:
import pandas as pd
img_list = [f'{i}.jpg' for i in index_list.cpu()]
dataframe = pd.DataFrame({'Id':img_list,'Category':predict_list.cpu()})
dataframe.to_csv("test.csv",index=False,sep=',')

In [16]:
import moxing as mox
mox.file.copy_parallel("test.csv","obs://deep-learning-hw2-zzzzzzjjjbbb/test.csv")